## TPC 2 - Conversor de MarkDown para HTML

Criar em Python um pequeno conversor de MarkDown para HTML para os elementos descritos na "Basic Syntax" da Cheat Sheet:


In [4]:
import re

### Cabeçalhos: linhas iniciadas por "# texto", ou "## texto" ou "### texto"

In: `# Exemplo`

Out: `<h1>Exemplo</h1>`


In [8]:
exemplo_1 = "# Hello"
exemplo_2 = "## My"
exemplo_3 = "### Friend"

def troca_titulo(texto):
    rc1 = re.compile(r"(#+)\s+(.+)")
    if len(rc1.match(texto).group(1)) == 1:
        return rc1.sub(r"<h1>\2</h1>",texto)
    elif len(rc1.match(texto).group(1)) == 2:
        return rc1.sub(r"<h2>\2</h2>",texto)
    else:
        return rc1.sub(r"<h3>\2</h3>",texto)

print(troca_titulo(exemplo_1))
print(troca_titulo(exemplo_2))
print(troca_titulo(exemplo_3))

<h1>Hello</h1>
<h2>My</h2>
<h3>Friend</h3>


### Bold: pedaços de texto entre "**":

In: `Este é um **exemplo** ...`

Out: `Este é um <b>exemplo</b> ...`

### Itálico: pedaços de texto entre "*":

In: `Este é um *exemplo* ...`

Out: `Este é um <i>exemplo</i> ...`

In [15]:
exemplo_1 = "Este é um **exemplo** ..."
exemplo_2 = "Este é um *exemplo* ..."
exemplo_3 = "Este é um *****exemplo*** ..."


def troca_bi(texto):
    rc2 = re.compile(r"(\*+)(.+?)(\*+)")
    if len(rc2.search(texto).group(1)) == len(rc2.search(texto).group(3)):
        if len(rc2.search(texto).group(1)) == 1:
            return rc2.sub(r"<i>\2</i>",texto)
        elif len(rc2.search(texto).group(1)) == 2:
            return rc2.sub(r"<b>\2</b>",texto)
        else:
            return texto
    else:
        return texto

print(troca_bi(exemplo_1))
print(troca_bi(exemplo_2))
print(troca_bi(exemplo_3))

Este é um <b>exemplo</b> ...
Este é um <i>exemplo</i> ...
Este é um *****exemplo*** ...



### Lista numerada:

In:
```
1. Primeiro item
2. Segundo item
3. Terceiro item
```

Out:
```
<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>
```


In [16]:
exemplo = """1. Primeiro item
2. Segundo item
3. Terceiro item"""

def troca_lista(texto):
    linhas = texto.splitlines()
    itens = []
    for linha in linhas:
        m = re.match(r"\d+\.\s+(.+)", linha)
        if m:
            itens.append(f"<li>{m.group(1)}</li>")
        else:
            itens.append(linha)
    
    return "<ol>\n" + "\n".join(itens) + "\n</ol>"

print(troca_lista(exemplo))

<ol>
<li>Primeiro item</li>
<li>Segundo item</li>
<li>Terceiro item</li>
</ol>


### Link: [texto](endereço URL)

In: `Como pode ser consultado em [página da UC](http://www.uc.pt)`

Out: `Como pode ser consultado em <a href="http://www.uc.pt">página da UC</a>`



In [29]:
exemplo = r"Como pode ser consultado em [página da UC](http://www.uc.pt)"

def troca_link(texto):
    rc4 = re.compile(r"\[(.+?)\]\((.+?)\)")
    return rc4.sub(r"<a href=\"\2\">\1</a>", texto)

print(troca_link(exemplo))

Como pode ser consultado em <a href=\"http://www.uc.pt\">página da UC</a>


### Imagem: ![texto alternativo](path para a imagem)

In: `Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ...`

Out: `Como se vê na imagem seguinte: <img src="http://www.coellho.com" alt="imagem dum coelho"/> ...`

In [31]:
exemplo = r"Como se vê na imagem seguinte: ![imagem dum coelho](http://www.coellho.com) ..."

def troca_imagem(texto):
    rc5 = re.compile(r"\!\[(.+?)\]\((.+?)\)")
    return rc5.sub(r"<img src=\"\2\"alt=\"\1\"/>", texto)

print(troca_imagem(exemplo))

Como se vê na imagem seguinte: <img src=\"http://www.coellho.com\"alt=\"imagem dum coelho\"/> ...
